In [ ]:
import numpy as np
import time
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
model_path = "/dataset/crosspipe/OriginModel/Llama-2-7b-chat-hf/"
device = "cuda:3" if torch.cuda.is_available() else "cpu"
model = LlamaForCausalLM.from_pretrained(model_path, device_map=device)
tokenizer = LlamaTokenizer.from_pretrained(model_path)

In [ ]:
input_text = "Once upon a time"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

In [ ]:
input_text = "Once upon a time"
inputs = tokenizer(input_text, return_tensors="pt").to(device)
# 生成文本并保存 KV 缓存
outputs = model.generate(**inputs, max_new_tokens=50, use_cache=True, return_dict_in_generate=True, output_scores=True)

# # 保存 KV 缓存

In [ ]:
outputs

In [ ]:
generated_ids = outputs.sequences

In [ ]:
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

print(generated_text)

In [53]:
past_key_values = outputs.past_key_values

In [57]:
past_key_values[0][0].shape

54

In [56]:
# 继续生成文本
input_text_continued = "And they lived happily ever after."
inputs_continued = tokenizer(input_text_continued, return_tensors="pt").to(device)
inputs_continued

{'input_ids': tensor([[    1,  1126,   896, 10600,  2250,  2354,  3926,  1156, 29889]],
       device='cuda:3'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:3')}

In [ ]:
# 继续生成文本并传递以前的 KV 缓存
outputs_continued = model.generate(
    **inputs_continued,
    max_new_tokens=50,
    use_cache=True,
    return_dict_in_generate=True,
    output_scores=True,
    past_key_values=past_key_values  # 传递以前的 KV 缓存
)

# 解码生成的文本
generated_text = tokenizer.decode(outputs_continued.sequences[0], skip_special_tokens=True)
print(generated_text)